# Linear Regression with PyTorch

Discusses one of the foundational algorithms of machine learning in this tutorial of Linear regression. We'll create a model that predicts crop yields for apples and oranges (**target variables**) by looking at the average temperature, rainfall and humidity (**input variables** or **features**) in a region. Here's the training data:

![Dataset to be used](./Images/1.png "Dataset")

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

\begin{equation*}
yield_{apple}  = w_{11} * temp + w_{12} * rainfall + w_{13} * humidity + b_1
\end{equation*}
\begin{equation*}
yield_{orange} = w_{21} * temp + w_{22} * rainfall + w_{23} * humidity + b_2
\end{equation*}

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![Visual Representation](./Images/2.png "Visualization")

The learning part of linear regression is to figure out a set of weights <code>w11, w12,... w23, b1 & b2</code> by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and humidity). This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called **gradient descent**.




In [1]:
import numpy as np
import torch

## Training data
The training data can be represented using 2 matrices: <code>inputs</code> and <code>targets</code>, each with one row per observation, and one column per variable.



In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

We've separated the input and target variables, because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some <code>CSV files</code> as <code>numpy arrays</code>, do some processing, and then convert them to <code>PyTorch tensors</code> as follows:

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch
The weights and biases (<code>w11, w12,... w23, b1 & b2</code>) can also be represented as matrices, initialized as random values. The first row of w and the first element of <code>b</code> are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.


In [5]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-1.8199, -0.3164, -0.6350],
        [ 0.6504, -0.1634,  0.4168]], requires_grad=True)
tensor([-0.2538, -0.0613], requires_grad=True)


<code>torch.randn</code> creates a tensor with the given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviation 1.

Our **model** is simply a function that performs a matrix multiplication of the inputs and the weights <code>w</code> (transposed) and adds the bias <code>b</code> (replicated for each observation).

![Equation1](./Images/3.png "Equation3")

We can define the model as follows:


In [6]:
def model(x):
    return x @ w.t() + b

<code>@</code> represents matrix multiplication in PyTorch, and the <code>.t</code> method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [7]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-181.6105,   54.3942],
        [-234.3481,   71.4237],
        [-237.8113,   58.8071],
        [-222.9853,   74.6742],
        [-200.6508,   58.3099]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [8]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see that there's a huge difference between the predictions of our model, and the actual values of the target variables. Obviously, this is because we've initialized our model with random weights and biases, and we can't expect it to *just work*.

### Loss function
Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

* Calculate the difference between the two matrices (preds and targets).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [9]:
# Mean Squared Loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

<code>torch.sum</code> returns the sum of all the elements in a tensor, and the <code>.numel</code> method returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [10]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(44716.4648, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by about 145 (square root of the loss 20834)*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. Also, the result is called the loss, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

In [11]:
# Compute gradients
loss.backward()

### Compute gradients
With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have <code>requires_grad</code> set to <code>True</code>.

In [12]:
# Gradients for the weights
print(w)
print(w.grad)

tensor([[-1.8199, -0.3164, -0.6350],
        [ 0.6504, -0.1634,  0.4168]], requires_grad=True)
tensor([[-24605.0488, -26233.6211, -16282.9199],
        [ -2126.0591,  -3559.2812,  -1944.2960]])


The gradients are stored in the <code>.grad</code> property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix, with the same dimensions.

In [13]:
print(b)
print(b.grad)

tensor([-0.2538, -0.0613], requires_grad=True)
tensor([-291.6812,  -28.4782])


The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the element's value slightly will **increase** the loss.
* **decreasing** the element's value slightly will **decrease** the loss.

![Positive](./Images/4.png "Positive")

If a gradient element is **negative**:

* **increasing** the element's value slightly will **decrease** the loss.
* **decreasing** the element's value slightly will **increase** the loss.

![Negative](./Images/5.png "Negative")

The increase or decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimization algorithm that we'll use to improve our model.



Before we proceed, we reset the gradients to zero by calling <code>.zero_()</code> method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call <code>.backward</code> on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results.

In [14]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


### Adjust weights and biases using gradient descent
We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

Let's implement the above step by step.



In [15]:
preds = model(inputs)
print(preds)

tensor([[-181.6105,   54.3942],
        [-234.3481,   71.4237],
        [-237.8113,   58.8071],
        [-222.9853,   74.6742],
        [-200.6508,   58.3099]], grad_fn=<AddBackward0>)


Note that the predictions are same as before, since we haven't made any changes to our model. The same holds true for the loss and gradients.

In [16]:
loss = mse(preds, targets)
print(loss)

tensor(44716.4648, grad_fn=<DivBackward0>)


In [17]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-24605.0488, -26233.6211, -16282.9199],
        [ -2126.0591,  -3559.2812,  -1944.2960]])
tensor([-291.6812,  -28.4782])


Finally, we update the weights and biases using the gradients computed above.



In [18]:
# Adjust weights & Reset gradients
with torch.no_grad():
    w -= w.grad *1e-5
    b -= b.grad *1e-5
    w.grad.zero_()
    b.grad.zero_()

A few things to note above:

* We use <code>torch.no_grad</code> to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weights and biases. 
* We multiply the gradients with a really small number (<code>10^-5</code> in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the learning rate of the algorithm. 
* After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations.

Let's take a look at the new weights and biases.

In [19]:
print(w)
print(b)

tensor([[-1.5739, -0.0540, -0.4722],
        [ 0.6716, -0.1278,  0.4363]], requires_grad=True)
tensor([-0.2508, -0.0610], requires_grad=True)


With the new weights and biases, the model should have lower loss.

In [20]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(30332.4590, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient descent.

### Train for multiple epochs
To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [21]:
for i in range(300):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [22]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(22.6262, grad_fn=<DivBackward0>)


As you can see, the loss is now much lower than what we started out with. Let's look at the model's predictions and compare them with the targets.

In [23]:
preds

tensor([[ 57.1763,  72.0968],
        [ 79.5908, 100.3065],
        [124.6241, 130.9111],
        [ 21.1122,  47.3118],
        [ 97.3047, 112.4836]], grad_fn=<AddBackward0>)

In [24]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Linear regression using PyTorch built-ins

The model and training process above were implemented using basic matrix operations. But since this such a common pattern , PyTorch has several built-in functions and classes to make it easy to create and train models.

Let's begin by importing the <code>torch.nn</code> package from PyTorch, which contains utility classes for building neural networks.



In [25]:
import torch.nn as nn

As before, we represent the inputs and targets and matrices.

In [44]:
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58],
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

targets = np.array([[56, 70], [81, 101], [119, 133],
                   [22, 37], [103, 119], [56, 70], 
                   [81, 101], [119, 133], [22, 37], 
                   [103, 119], [56, 70], [81, 101], 
                   [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

We are using 15 training examples this time, to illustrate how to work with large datasets in small batches.

### Dataset and DataLoader
We'll create a <code>TensorDataset</code>, which allows access to rows from <code>inputs</code> and <code>targets</code> as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [45]:
from torch.utils.data import TensorDataset

In [46]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The <code>TensorDataset</code> allows us to access a small section of the training data using the array indexing notation (<code>[0:3]</code> in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a <code>DataLoader</code>, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [47]:
from torch.utils.data import DataLoader

In [48]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle = True)

The data loader is typically used in a <code>for-in</code> loop. Let's look at an example.

In [49]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 87., 134.,  58.],
        [ 91.,  88.,  64.]])
tensor([[119., 133.],
        [119., 133.],
        [ 56.,  70.],
        [119., 133.],
        [ 81., 101.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If <code>shuffle</code> is set to <code>True</code>, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

### nn.Linear
Instead of initializing the weights & biases manually, we can define the model using the <code>nn.Linear</code> class from PyTorch, which does it automatically.



In [50]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3747,  0.1607,  0.4104],
        [ 0.5118, -0.1949, -0.2882]], requires_grad=True)
Parameter containing:
tensor([-0.4002, -0.1280], requires_grad=True)


PyTorch models also have a helpful <code>.parameters</code> method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [51]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.3747,  0.1607,  0.4104],
         [ 0.5118, -0.1949, -0.2882]], requires_grad=True),
 Parameter containing:
 tensor([-0.4002, -0.1280], requires_grad=True)]

We can use the model to generate predictions in the exact same way as before:

In [52]:
preds = model(inputs)
preds

tensor([[  0.6596,  11.7803],
        [  5.9076,  10.8474],
        [ 12.3380,   1.5641],
        [-16.5274,  33.0277],
        [ 17.8998,  -3.6993],
        [  0.6596,  11.7803],
        [  5.9076,  10.8474],
        [ 12.3380,   1.5641],
        [-16.5274,  33.0277],
        [ 17.8998,  -3.6993],
        [  0.6596,  11.7803],
        [  5.9076,  10.8474],
        [ 12.3380,   1.5641],
        [-16.5274,  33.0277],
        [ 17.8998,  -3.6993]], grad_fn=<AddmmBackward>)

### Loss Function
Instead of defining a loss function manually, we can use the built-in loss function <code>mse_loss</code>.

In [53]:
# import nn.functional
import torch.nn.functional as F

The <code>nn.functional</code> package contains many useful loss functions and several other utilities.

In [54]:
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [55]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(7266.7939, grad_fn=<MseLossBackward>)


### Optimizer
Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer <code>optim.SGD</code>. SGD stands for <code>stochastic gradient descent</code>. It is called <code>stochastic</code> because samples are selected in batches (often with random shuffling) instead of as a single group.

In [56]:
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

Note that <code>model.parameters()</code> is passed as an argument to <code>optim.SGD</code>, so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which the parameters are modified.

### Train the model
We are now ready to train the model. We'll follow the exact same process to implement gradient descent:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function <code>fit</code> which trains the model for a given number of epochs.

In [57]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    '''
    fit model functions
    '''
    for epoch in range(num_epochs):
        #Train with batches of data
        for xb, yb in train_dl:
            
            #1. Generaate predictions
            pred = model(xb)
            
            #2. Calculate loss
            loss = loss_fn(pred, yb)
            
            #3. Compute gradients
            loss.backward()
            
            #4. Adjust/Update the weights using gradients
            opt.step()
            
            #5. Reset the gradients
            opt.zero_grad()
            
        #Print progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use <code>opt.step</code> to perform the update, and <code>opt.zero_grad</code> to reset the gradients to zero.

* We've also added a log statement which prints the loss from the last batch of data for every 10th epoch, to track the progress of training. <code>loss.item</code> returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.



In [58]:
fit(300, model, loss_fn, opt)

Epoch [10/300], Loss: 480.9478
Epoch [20/300], Loss: 348.4428
Epoch [30/300], Loss: 111.8379
Epoch [40/300], Loss: 216.1943
Epoch [50/300], Loss: 176.2247
Epoch [60/300], Loss: 92.3112
Epoch [70/300], Loss: 64.1524
Epoch [80/300], Loss: 67.2938
Epoch [90/300], Loss: 26.2885
Epoch [100/300], Loss: 49.2348
Epoch [110/300], Loss: 41.2469
Epoch [120/300], Loss: 23.8804
Epoch [130/300], Loss: 37.1890
Epoch [140/300], Loss: 21.2262
Epoch [150/300], Loss: 26.9534
Epoch [160/300], Loss: 16.4480
Epoch [170/300], Loss: 15.2440
Epoch [180/300], Loss: 21.3798
Epoch [190/300], Loss: 14.2732
Epoch [200/300], Loss: 21.2338
Epoch [210/300], Loss: 4.4456
Epoch [220/300], Loss: 15.9523
Epoch [230/300], Loss: 7.8090
Epoch [240/300], Loss: 9.4623
Epoch [250/300], Loss: 10.4808
Epoch [260/300], Loss: 13.1041
Epoch [270/300], Loss: 6.3254
Epoch [280/300], Loss: 17.6521
Epoch [290/300], Loss: 10.9339
Epoch [300/300], Loss: 9.8862


Let's generate predictions using our model and verify that they're close to our targets.

In [59]:
preds = model(inputs)
preds

tensor([[ 57.1159,  70.5643],
        [ 82.2156,  97.9476],
        [118.7582, 138.2696],
        [ 21.1821,  38.7057],
        [101.7977, 113.4236],
        [ 57.1159,  70.5643],
        [ 82.2156,  97.9476],
        [118.7582, 138.2696],
        [ 21.1821,  38.7057],
        [101.7977, 113.4236],
        [ 57.1159,  70.5643],
        [ 82.2156,  97.9476],
        [118.7582, 138.2696],
        [ 21.1821,  38.7057],
        [101.7977, 113.4236]], grad_fn=<AddmmBackward>)

In [60]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Indeed, the predictions are quite close to our targets, and now we have a fairly good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall and humidity in a region.

# Further Reading
Covered a lot of ground this this tutorial, including *linear regression* and the *gradient descent optimization* algorithm. Here are a few resources if you'd like to dig deeper into these topics:

* For a more detailed explanation of derivates and gradient descent, see [these notes from a Udacity course](https://storage.googleapis.com/supplemental_media/udacityu/315142919/Gradient%20Descent.pdf).

* For an animated visualization of how linear regression works, [see this post](https://hackernoon.com/visualizing-linear-regression-with-pytorch-9261f49edb09).

* For a more mathematical treatment of matrix calculus, linear regression and gradient descent, you should check out [Andrew Ng's excellent course notes](https://github.com/cleor41/CS229_Notes/blob/master/lectures/cs229-notes1.pdf) from CS229 at Stanford University.

* To practice and test your skills, you can participate in the [Boston Housing Price Prediction](https://www.kaggle.com/c/boston-housing) competition on Kaggle, a website that hosts data science competitions.

With this, we complete our discussion of linear regression in PyTorch, and we’re ready to move on to the next topic: *Logistic regression*.



--------